# Initial Recipe Dataset Parsing and Preparation

### Import dependencies

In [25]:
import json
import pandas as pd

### Read in original recipe dataset

In [26]:
data_path = "orig_data/full_dataset.csv"
recipe_df = pd.read_csv(data_path)

### Dataset Exploration

In [27]:
recipe_df.head()

,Unnamed: 0,title,ingredients,directions,link,source,NER
0,0,No-Bake Nut Cookies,"[""1 c. firmly packed brown sugar"", ""1/2 c. eva...","[""In a heavy 2-quart saucepan, mix brown sugar...",www.cookbooks.com/Recipe-Details.aspx?id=44874,Gathered,"[""brown sugar"", ""milk"", ""vanilla"", ""nuts"", ""bu..."
1,1,Jewell Ball'S Chicken,"[""1 small jar chipped beef, cut up"", ""4 boned ...","[""Place chipped beef on bottom of baking dish....",www.cookbooks.com/Recipe-Details.aspx?id=699419,Gathered,"[""beef"", ""chicken breasts"", ""cream of mushroom..."
2,2,Creamy Corn,"[""2 (16 oz.) pkg. frozen corn"", ""1 (8 oz.) pkg...","[""In a slow cooker, combine all ingredients. C...",www.cookbooks.com/Recipe-Details.aspx?id=10570,Gathered,"[""frozen corn"", ""cream cheese"", ""butter"", ""gar..."
3,3,Chicken Funny,"[""1 large whole chicken"", ""2 (10 1/2 oz.) cans...","[""Boil and debone chicken."", ""Put bite size pi...",www.cookbooks.com/Recipe-Details.aspx?id=897570,Gathered,"[""chicken"", ""chicken gravy"", ""cream of mushroo..."
4,4,Reeses Cups(Candy),"[""1 c. peanut butter"", ""3/4 c. graham cracker ...","[""Combine first four ingredients and press in ...",www.cookbooks.com/Recipe-Details.aspx?id=659239,Gathered,"[""peanut butter"", ""graham cracker crumbs"", ""bu..."


In [28]:
print(recipe_df.iloc[0]["NER"])
print(recipe_df.iloc[0]["directions"])

["brown sugar", "milk", "vanilla", "nuts", "butter", "bite size shredded rice biscuits"]
["In a heavy 2-quart saucepan, mix brown sugar, nuts, evaporated milk and butter or margarine.", "Stir over medium heat until mixture bubbles all over top.", "Boil and stir 5 minutes more. Take off heat.", "Stir in vanilla and cereal; mix well.", "Using 2 teaspoons, drop and shape into 30 clusters on wax paper.", "Let stand until firm, about 30 minutes."]


### Set up small test dataset 

To be used for shorter integration testing

In [21]:
short_recipe_df = recipe_df.iloc[:5]

In [ ]:
print(short_recipe_df)

In [29]:
for index, row in short_recipe_df.iterrows():
    entities = row['NER']
    entity_list = json.loads(entities)
    print("ingredients-----------------------")
    ingredients = row["ingredients"]
    print(ingredients)
    print("directions------------------------")
    directions = row["directions"]
    print(directions)

    for entity in entity_list:
        try:
            start_index = directions.index(entity)
            end_index = start_index + len(entity)
        except:
            print("Warning, following entity not found----------")
            print(entity)


ingredients-----------------------
["1 c. firmly packed brown sugar", "1/2 c. evaporated milk", "1/2 tsp. vanilla", "1/2 c. broken nuts (pecans)", "2 Tbsp. butter or margarine", "3 1/2 c. bite size shredded rice biscuits"]
directions------------------------
["In a heavy 2-quart saucepan, mix brown sugar, nuts, evaporated milk and butter or margarine.", "Stir over medium heat until mixture bubbles all over top.", "Boil and stir 5 minutes more. Take off heat.", "Stir in vanilla and cereal; mix well.", "Using 2 teaspoons, drop and shape into 30 clusters on wax paper.", "Let stand until firm, about 30 minutes."]
Warning, following entity not found----------
bite size shredded rice biscuits
ingredients-----------------------
["1 small jar chipped beef, cut up", "4 boned chicken breasts", "1 can cream of mushroom soup", "1 carton sour cream"]
directions------------------------
["Place chipped beef on bottom of baking dish.", "Place chicken on top of beef.", "Mix soup and cream together; pour

#### Lessons learned from small test
* Can not use context for training from directions, must use ingredient description list
* Reason: main listed ingredient entity may not exist in the directions

### Pre-processing of recipe training data 
Convert original dataframe to csv for next step - spacy v3 formatting

In [30]:
training_data = []
with open('recipe_training_data_v3.csv', 'w') as file:
   # file.write('[')
    for index, row in recipe_df.iterrows():
        if(index % 10000 == 0):
            print(index)
        entity_list = []
        entities = row['NER']
        entity_list = json.loads(entities)
        #print("entities--------------------------")
        #print(entity_list)
        
        #print("ingredients-----------------------")
        ingredient_descs = row["ingredients"]#["directions"]
        #print(ingredients)
        
        #print("directions------------------------")
        directions = row["directions"]
        #print(directions)
        
        ingredient_desc_list = json.loads(ingredient_descs)

        # Remove errors where ingredient description list does not match 
        # entity list from a mis-labeling
        if (len(ingredient_desc_list) == len(entity_list)):
            
            ingredient_data_list = [] 
            for index2, ingredient_desc in enumerate(ingredient_desc_list):

                entity = entity_list[index2]
                try:
                    entity_dic = {}
                    #print("ingredient descr")
                    #print(ingredient_desc)
                    #print('entity')
                    #print(entity)
            
                    start_index = ingredient_desc.index(entity)
                    end_index = start_index + len(entity)
                    entity_info = [(start_index, end_index, entity)]
                    ingred_data = (ingredient_desc, entity_info)

                    file.write('{}\n'.format(ingred_data))

                except:
                    print("Warning, following entity not found----------")
                    print(entity)
                    # Todo: if time permits, remove @ symbols and do basic tokenization


0
10000
20000
30000
40000
50000
60000
70000
80000
90000
100000
110000
120000
130000
140000
150000
160000
170000
180000
190000
200000
210000
220000
Warning, following entity not found----------
sautéed onion
230000
240000
250000
260000
270000
280000
Warning, following entity not found----------
sautéed onion
290000
300000
310000
320000
330000
340000
350000
360000
370000
380000
390000
400000
410000
420000
430000
440000
450000
460000
470000
480000
490000
500000
510000
520000
530000
540000
550000
560000
570000
580000
590000
600000
610000
620000
630000
640000
650000
660000
670000
680000
690000
Warning, following entity not found----------
sautéed mushrooms
700000
710000
720000
730000
740000
750000
760000
770000
780000
790000
800000
810000
820000
830000
840000
850000
860000
870000
880000
890000
Warning, following entity not found----------
NILLA® waters
Warning, following entity not found----------
Swiss Miss®
Warning, following entity not found----------
Snack Pack®
Warning, following entit

Warning, following entity not found----------
TABASCO® Pepper Sauce
Warning, following entity not found----------
Barilla® Pronto
Warning, following entity not found----------
Campbell's®
Warning, following entity not found----------
Campbell's®
Warning, following entity not found----------
'S®
Warning, following entity not found----------
Wish-Bone®
Warning, following entity not found----------
Barilla®
Warning, following entity not found----------
HORMEL®
Warning, following entity not found----------
™
Warning, following entity not found----------
Campbell's®
Warning, following entity not found----------
Bee®
Warning, following entity not found----------
KITCHENS®
Warning, following entity not found----------
Rice®
Warning, following entity not found----------
All Natural Golden Brown® Mild Sausage
Warning, following entity not found----------
Progresso® lemon pepper
Warning, following entity not found----------
Barilla® Pesto
Warning, following entity not found----------
Campbell's®

Warning, following entity not found----------
™
Warning, following entity not found----------
Campbell's®
Warning, following entity not found----------
Campbell's®
Warning, following entity not found----------
VELVEETA®
Warning, following entity not found----------
Campbell's®
Warning, following entity not found----------
SPLENDA® Granular
Warning, following entity not found----------
Barilla®
Warning, following entity not found----------
™
Warning, following entity not found----------
Wish-Bone®
Warning, following entity not found----------
'S® Chipits
Warning, following entity not found----------
'S® Chipits
Warning, following entity not found----------
Campbell's®
Warning, following entity not found----------
Snack Pack®
Warning, following entity not found----------
NO YOLKS®
Warning, following entity not found----------
Snack Pack®
Warning, following entity not found----------
Bee®
Warning, following entity not found----------
Campbell's®
Warning, following entity not found--------

1250000
Warning, following entity not found----------
tomato purée
Warning, following entity not found----------
tomato purée
Warning, following entity not found----------
crème fraîche
Warning, following entity not found----------
tomato purée
Warning, following entity not found----------
glacé cherries
Warning, following entity not found----------
tomato purée
Warning, following entity not found----------
Gruyère cheese
Warning, following entity not found----------
Gruyère
Warning, following entity not found----------
Lime Aïoli
Warning, following entity not found----------
crème fraîche
Warning, following entity not found----------
glacé cherries
Warning, following entity not found----------
frisée lettuce leaves
Warning, following entity not found----------
Crème fraîche
Warning, following entity not found----------
tomato purée
Warning, following entity not found----------
puréed
Warning, following entity not found----------
Gruyère cheese
Warning, following entity not found------

Warning, following entity not found----------
jalapeños
Warning, following entity not found----------
crème fraîche
Warning, following entity not found----------
Crème Fraîche
Warning, following entity not found----------
crème fraîche
Warning, following entity not found----------
crème fraîche
Warning, following entity not found----------
Gruyère
Warning, following entity not found----------
’s cheese
Warning, following entity not found----------
¼
Warning, following entity not found----------
crème fraîche
Warning, following entity not found----------
tomato purée
Warning, following entity not found----------
crème fraîche
Warning, following entity not found----------
tomato purée
Warning, following entity not found----------
crème fraîche
Warning, following entity not found----------
tomato purée
Warning, following entity not found----------
crème fraîche
Warning, following entity not found----------
crème fraîche
Warning, following entity not found----------
tomato purée
Warning, f

...

Warning, following entity not found----------
Jif® Natural Creamy
Warning, following entity not found----------
¼
Warning, following entity not found----------
crème fraîche
Warning, following entity not found----------
Rhône
Warning, following entity not found----------
Recipe Secrets®
Warning, following entity not found----------
’s Natural
Warning, following entity not found----------
Salad® Dressing
Warning, following entity not found----------
Recipe Secrets®
Warning, following entity not found----------
chicken stock Kitchen Basics® Original
Warning, following entity not found----------
pimentón
Warning, following entity not found----------
Comment®
Warning, following entity not found----------
gruyère cheese
Warning, following entity not found----------
’s
Warning, following entity not found----------
tomato purée
Warning, following entity not found----------
tomato purée
Warning, following entity not found----------
¼
Warning, following entity not found----------
peas – frozen 

Warning, following entity not found----------
crème fraiche
Warning, following entity not found----------
Gruyère cheese
Warning, following entity not found----------
’s
Warning, following entity not found----------
® Classic
Warning, following entity not found----------
¼
Warning, following entity not found----------
’s sugar
Warning, following entity not found----------
Fisher Chef’s Naturals
Warning, following entity not found----------
®
Warning, following entity not found----------
very ﬁnely ground espresso beans
Warning, following entity not found----------
Recipe Secrets®
Warning, following entity not found----------
Recipe Secrets®
Warning, following entity not found----------
¼
Warning, following entity not found----------
¼
Warning, following entity not found----------
½oz
Warning, following entity not found----------
italian sausages –
Warning, following entity not found----------
• fresh
Warning, following entity not found----------
tomato purée
Warning, following entity n

Warning, following entity not found----------
Gruyére
Warning, following entity not found----------
crème de cassis
Warning, following entity not found----------
sherry ¼
Warning, following entity not found----------
’s worth
Warning, following entity not found----------
ginger McCormick®
Warning, following entity not found----------
caribbean jerk seasoning McCormick® Perfect Pinch®
Warning, following entity not found----------
chili powder McCormick®
Warning, following entity not found----------
¼
Warning, following entity not found----------
¼
Warning, following entity not found----------
¼
Warning, following entity not found----------
chicken sausages Gilbert’s Caprese
Warning, following entity not found----------
crème fraîche
Warning, following entity not found----------
¼
Warning, following entity not found----------
chili jalapeño
Warning, following entity not found----------
tomato purée
Warning, following entity not found----------
frozen açaí
Warning, following entity not fo

Warning, following entity not found----------
crème fraiche
Warning, following entity not found----------
pumpkin purée
Warning, following entity not found----------
sugar Baker’s
Warning, following entity not found----------
sugar Baker’s
Warning, following entity not found----------
crème fraîche
Warning, following entity not found----------
OREO® Cookies
Warning, following entity not found----------
tomato purée
Warning, following entity not found----------
ginger McCormick®
Warning, following entity not found----------
¼
Warning, following entity not found----------
Recipe Secrets®
Warning, following entity not found----------
chèvre
Warning, following entity not found----------
Recipe Secrets®
Warning, following entity not found----------
Recipe Secrets®
Warning, following entity not found----------
Recipe Secrets®
1630000
Warning, following entity not found----------
gruyère cheese
Warning, following entity not found----------
lettuce leaves some frisée
Warning, following entity 

Warning, following entity not found----------
cherry glacé
Warning, following entity not found----------
’s sugar
Warning, following entity not found----------
®
Warning, following entity not found----------
cottage cheese ½%%
Warning, following entity not found----------
crème fraiche
Warning, following entity not found----------
tomato purée
1640000
Warning, following entity not found----------
pumpkin purée store-bought
Warning, following entity not found----------
jalapeños
Warning, following entity not found----------
breading Shore Lunch® Chicken
Warning, following entity not found----------
Wish-Bone® Robusto
Warning, following entity not found----------
Bisquick® baking mix
Warning, following entity not found----------
crème fraîche
Warning, following entity not found----------
filé powder
Warning, following entity not found----------
crème fraîche
Warning, following entity not found----------
®
Warning, following entity not found----------
crème fraîche
Warning, following enti